In [ ]:
import pandas as pd
import re
from jiwer import wer
from jiwer import compute_measures
import ast


## Load Results

In [ ]:
kelechi_phi_whisper = pd.read_excel('/home/kelechi/bio_ramp_asr/results/bio_ramp_asr_results.xlsx',engine='openpyxl')
ashisa_results_1 = pd.read_csv('/home/kelechi/bio_ramp_asr/results/afrispeech_asr - afrispeech_asr.csv')
aisha_results_2 = pd.read_csv('/home/kelechi/bio_ramp_asr/results/us_medical_20_asr - us_medical_20_asr.csv')

# concat the two results
aisha_granite_parakeet = pd.concat([ashisa_results_1, aisha_results_2], ignore_index=True, sort=False)

evaluate_data = pd.merge(kelechi_phi_whisper, aisha_granite_parakeet[['utterance_id', 'Nvidia-Parakeet', 'IBM-Granite']], on='utterance_id', how='inner')
evaluate_data['Phi-4-ASR'] = evaluate_data['Phi-4-ASR'].apply(lambda x: pd.NA if isinstance(x, str) and "ERROR: CUDA out of memory" in x else x)

## Text Normalize

In [ ]:
def remove_timestamps(text: str) -> str:
    """
    Remove timestamps matching patterns like:
      - 03:18:98
      - 00:00:001
    (pattern: 1-2 digits ':' 1-2 digits ':' 1-3 digits)

    Returns cleaned string with extra spaces collapsed.
    """
    if not isinstance(text, str):
        return text
    # remove timestamp tokens
    cleaned = re.sub(r'\b\d{1,2}:\d{1,2}:\d{1,3}\b', '', text)
    # remove new line characters (convert to spaces), then collapse multiple spaces and trim
    cleaned = cleaned.replace('\n', ' ').replace('\r', ' ')
    cleaned = re.sub(r'\s+', ' ', cleaned).strip()
    
    # remove speaker tags like: [Speaker 1]:  Speaker 1:  speaker1:  D:  P:
    cleaned = re.sub(r'\[?[Ss]peaker\s*\d+\]?:', '', cleaned)
    cleaned = re.sub(r'\b[Dd]:', '', cleaned)
    cleaned = re.sub(r'\b[Pp]:', '', cleaned)
    
    # remove all characters that is not A-Z, a-z, 0-9 or space
    cleaned = re.sub(r"[^a-zA-Z0-9\s]", "", cleaned)
    
    # lowercase the text
    cleaned = cleaned.lower()
    return cleaned


In [ ]:
# apply remove_timestamps to the human transcript column
evaluate_data['norm_human_transcript'] = evaluate_data['human-transcript'].apply(remove_timestamps)
evaluate_data['norm_whisper_asr'] = evaluate_data['Whisper-ASR'].apply(remove_timestamps)
evaluate_data['norm_phi4_asr'] = evaluate_data['Phi-4-ASR'].apply(remove_timestamps)
evaluate_data['norm_parakeet'] = evaluate_data['Nvidia-Parakeet'].apply(remove_timestamps)
evaluate_data['norm_granite'] = evaluate_data['IBM-Granite'].apply(remove_timestamps)

## Calculate WER

In [ ]:
norm_columns = ['norm_whisper_asr', 'norm_phi4_asr', 'norm_parakeet', 'norm_granite']
# Calculate WER for each ASR column
for norm_col in norm_columns:
    evaluate_data[f'{norm_col}_wer_compute'] = evaluate_data.apply(
        lambda row: compute_measures(
            row['norm_human_transcript'] if pd.notna(row['norm_human_transcript']) else "",
            row[norm_col] if pd.notna(row[norm_col]) else ""
        ), axis=1
    )
    evaluate_data[f'{norm_col}_wer'] = evaluate_data[f'{norm_col}_wer_compute'].apply(lambda measures: measures['wer'])
    evaluate_data[f'{norm_col}_ins'] = evaluate_data[f'{norm_col}_wer_compute'].apply(
        lambda measures: measures['insertions']
    )
    evaluate_data[f'{norm_col}_del'] = evaluate_data[f'{norm_col}_wer_compute'].apply(
        lambda measures: measures['deletions']
    )
    evaluate_data[f'{norm_col}_sub'] = evaluate_data[f'{norm_col}_wer_compute'].apply(
        lambda measures: measures['substitutions']
    )
    evaluate_data[f'{norm_col}_ops'] = evaluate_data[f'{norm_col}_wer_compute'].apply(
        lambda measures: measures['ops']
    )
    

## Add Alignment for Sub, Del and Ins Words

In [ ]:
def preprocess_alignment_ops(alignment_ops):
    # If already a list/dict, return as-is (may be nested list)
    if isinstance(alignment_ops, (list, dict)):
        return alignment_ops
    if alignment_ops is None or (isinstance(alignment_ops, float) and pd.isna(alignment_ops)):
        return []
    s = str(alignment_ops)
    s = s.replace("AlignmentChunk(", "{").replace(")", "}").replace("type=", "'type':")\
         .replace("ref_start_idx=", "'ref_start_idx':").replace("ref_end_idx=", "'ref_end_idx':")\
         .replace("hyp_start_idx=", "'hyp_start_idx':").replace("hyp_end_idx=", "'hyp_end_idx':")
    if s.startswith("[[") and s.endswith("]]"):
        s = s[1:-1]
    return s

def _get_field(op, field, default=None):
    # support dict-like
    if isinstance(op, dict):
        return op.get(field, default)
    # support object with attribute (AlignmentChunk)
    if hasattr(op, field):
        return getattr(op, field, default)
    # support tuple/list with numeric positions (fallback not used here)
    return default

def extract_words_from_alignment(ref_text, hyp_text, alignment_ops):
    deletions, insertions, substitutions, equals = [], [], [], []
    ref_words = ref_text.split() if isinstance(ref_text, str) else []
    hyp_words = hyp_text.split() if isinstance(hyp_text, str) else []

    # if hypothesis empty -> all deleted
    if not hyp_words:
        return ref_words, [], [], []

    processed = preprocess_alignment_ops(alignment_ops)

    # produce a flat list of op entries regardless of input shape
    if isinstance(processed, list):
        # flatten one level (handles [[...]] case)
        flat = []
        for item in processed:
            if isinstance(item, list):
                flat.extend(item)
            else:
                flat.append(item)
        alignment_ops_list = flat
    elif isinstance(processed, dict):
        alignment_ops_list = [processed]
    else:
        if not processed:
            return deletions, insertions, equals, substitutions
        try:
            alignment_ops_list = ast.literal_eval(processed)
            # if parsed to nested list, flatten one level
            if isinstance(alignment_ops_list, list) and any(isinstance(x, list) for x in alignment_ops_list):
                flat = []
                for item in alignment_ops_list:
                    if isinstance(item, list):
                        flat.extend(item)
                    else:
                        flat.append(item)
                alignment_ops_list = flat
        except Exception as e:
            raise ValueError(f"Failed to parse alignment_ops for row: {e}")

    for op in alignment_ops_list:
        typ = _get_field(op, "type")
        if typ == "delete":
            s = _get_field(op, "ref_start_idx", 0)
            e = _get_field(op, "ref_end_idx", 0)
            deletions.extend(ref_words[s:e])
        elif typ == "equal":
            s = _get_field(op, "hyp_start_idx", 0)
            e = _get_field(op, "hyp_end_idx", 0)
            equals.extend(hyp_words[s:e])
        elif typ == "insert":
            s = _get_field(op, "hyp_start_idx", 0)
            e = _get_field(op, "hyp_end_idx", 0)
            insertions.extend(hyp_words[s:e])
        elif typ == "substitute":
            s = _get_field(op, "ref_start_idx", 0)
            e = _get_field(op, "ref_end_idx", 0)
            substitutions.extend(ref_words[s:e])

    return deletions, insertions, equals, substitutions

In [ ]:
# apply extract_words_from_alignment to each row in the DataFrame i.e Whisper ASR results and Phi-4 ASR results 

asr_rows = ['norm_whisper_asr', 'norm_phi4_asr', 'norm_parakeet', 'norm_granite']
align_ops_rows = ['norm_whisper_asr_ops', 'norm_phi4_asr_ops', 'norm_parakeet_ops', 'norm_granite_ops']

for asr_col, ops_col in zip(asr_rows, align_ops_rows):
    all_deletions = []
    all_insertions = []
    all_equals = []
    all_substitutions = []
    error_messages = []

    for index, row in evaluate_data.iterrows():
        try:
            result = extract_words_from_alignment(row['norm_human_transcript'], row[asr_col], row[ops_col])
            all_deletions.append(result[0])
            all_insertions.append(result[1])
            all_equals.append(result[2])
            all_substitutions.append(result[3])
            error_messages.append("")
        except Exception as e:
            all_deletions.append([])
            all_insertions.append([])
            all_equals.append([])
            all_substitutions.append([])
            error_messages.append(str(e))
            print(f"Error processing row {index} for {asr_col}: {str(e)}")

    # Creating DataFrame for the extracted words and error messages
    extracted_words_df = pd.DataFrame({
        f'{asr_col}_Deletions': all_deletions,
        f'{asr_col}_Insertions': all_insertions,
        # f'{asr_col}_Equals': all_equals,
        f'{asr_col}_Substitutions': all_substitutions,
        # f'{asr_col}_Error_Message': error_messages
    })

    # Concatenate the extracted words DataFrame with the original DataFrame
    evaluate_data = pd.concat([evaluate_data, extracted_words_df], axis=1, join='outer')


In [ ]:
# save the dataframe to a new excel file
evaluate_data.to_excel('/home/kelechi/bio_ramp_asr/all_result_processed.xlsx', index=False, engine='openpyxl')